1. **Le Principe Fondamental : La "Table Infinie"**  
Le concept de Spark Structured Streaming est de traiter un flux de données non pas comme une séquence de paquets, mais comme une table qui grandit à l'infini (Unbounded Table).  
Concept : Chaque nouvelle donnée arrivant d'un capteur est simplement une nouvelle ligne ajoutée ("appended") à cette table virtuelle.
Utilité SmartTech : Cela nous permet d'utiliser le même code et la même logique SQL que pour nos analyses historiques (Batch). Pas besoin d'apprendre un nouveau langage pour le temps réel.  

2. **Lecture et Écriture de Flux**  
Contrairement au traitement par lots (Batch) qui a un début et une fin, le streaming est continu.
API : On utilise spark.readStream pour définir la source et df.writeStream pour démarrer le traitement.
Sources supportées :
- Kafka : Le standard pour ingérer nos millions de messages IoT.
- Files (Fichiers) : Pour surveiller l'arrivée de nouveaux fichiers CSV/JSON dans un dossier.
- Sinks (Destinations) supportés :
- Kafka : Pour renvoyer des alertes vers d'autres services.
- Delta Lake : Pour stocker l'historique fiable.
- Console/Memory : Pour le débogage.  

3. **Le Rythme du Traitement : Les Triggers**  
Le Trigger définit quand Spark doit vérifier s'il y a de nouvelles données et lancer le calcul.
ProcessingTime (Défaut) : "Vérifie toutes les X secondes". Idéal pour nos tableaux de bord de monitoring (latence ~100ms à quelques secondes).
Once / AvailableNow : "Traite tout ce qui est en attente, puis éteins-toi". Parfait pour réduire les coûts cloud si on n'a besoin de mettre à jour les données que toutes les heures.
Continuous (Expérimental) : Pour une latence ultra-faible (milliseconde), mais avec des limitations techniques.

4. **Gestion du Temps et des Retards (Windowing & Watermarks)**  
En IoT, les problèmes de réseau sont fréquents. Une donnée de 10h00 peut arriver à 10h05.
Windowing (Fenêtrage) : Permet d'agréger les données par tranches de temps (ex: "Moyenne de température par tranche de 5 minutes") plutôt que par heure d'arrivée technique.
Watermarks (Fil d'eau) : C'est le seuil de tolérance. On dit au système : "Accepte les données en retard jusqu'à 10 minutes. Au-delà, ignore-les pour ne pas surcharger la mémoire." C'est crucial pour la stabilité de nos serveurs.

5. **Modes de Sortie (Output Modes)**  
Comment mettre à jour le résultat quand de nouvelles données arrivent ?
Append (Ajout) : On ajoute seulement les nouvelles lignes.
Usage SmartTech : Stockage brut des messages des capteurs.
Update (Mise à jour) : On met à jour les lignes qui ont changé.
Usage SmartTech : Maintien de la "dernière température connue" pour chaque machine.
Complete (Complet) : On réécrit tout le tableau à chaque fois.
Usage SmartTech : Compteurs simples (ex: "Nombre total d'erreurs depuis le démarrage"), à éviter sur de gros volumes.

6. **Fiabilité : Checkpointing et Tolérance aux Pannes**  
C'est l'assurance-vie du pipeline de données.
Checkpointing : Spark sauvegarde régulièrement sa progression (offsets) et son état intermédiaire sur un stockage durable (HDFS/S3).
Garantie : En cas de crash du serveur, Spark redémarre, lit le checkpoint et reprend exactement là où il s'est arrêté. Cela garantit une sémantique "Exactly-once" (aucune donnée perdue, aucune donnée comptée deux fois), indispensable pour la facturation ou la maintenance prédictive.

7. **Architecture Médaillon (Best Practice)**  
Pour organiser ce flux de données, nous adoptons une architecture en couches :
Couche
Rôle chez SmartTech
Mode Spark
- BRONZE  
Zone d'atterrissage. Données brutes (Raw), ingérées depuis Kafka sans modification. Permet de "rejouer" l'histoire en cas de bug.
Append
- SILVER  
Zone de nettoyage. Données filtrées, dédupliquées et typées (Schéma validé). C'est la source de vérité propre.
Append / Update
- GOLD  
Zone métier. Données agrégées prêtes pour la consommation (KPIs, Moyennes horaires pour BI).
Complete / Update